In [15]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn import set_config; set_config(display='diagram')

In [16]:
train_filepath='../titanic/data/train.csv'
test_filepath='../titanic/data/test.csv'
titanic_data=pd.read_csv(train_filepath)
test_data=pd.read_csv(test_filepath)
df=titanic_data.copy()

In [18]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [3]:
def who(df):
    if df['Age']<16.0:
        return 'child'
    elif df['Sex']=='male':
        return 'male'
    elif df['Sex']=='female':
        return 'female'
df['who']=df.apply(who,axis=1)

In [4]:
df.drop(['PassengerId','Cabin','Name','Ticket','Sex'],axis=1,inplace=True)
df.dropna(axis=0, subset=['Embarked'],inplace=True)


In [5]:
num_feature=['Age','Fare']
cat_feature=['Pclass','SibSp','Parch','Embarked','who']
target=['Survived']

In [6]:
X=df.drop(columns=target)
y=df[target]

In [7]:
num_transformer = Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', RobustScaler())])

num_preprocessor = ColumnTransformer([('num_transformer',
                                                num_transformer,
                                                num_feature)])
num_preprocessor

ColumnTransformer(transformers=[('num_transformer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', RobustScaler())]),
                                 ['Age', 'Fare'])])

In [8]:
cat_ohe_transformer=Pipeline([('imputer',SimpleImputer(strategy='most_frequent')),
                              ('OHE',OneHotEncoder(sparse=False,handle_unknown='ignore'))
                                 ])

cat_preprocessor = ColumnTransformer([
                              ('cat_to_ohe',cat_ohe_transformer,cat_feature)
                              ],remainder='passthrough')
cat_preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('cat_to_ohe',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('OHE',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 ['Pclass', 'SibSp', 'Parch', 'Embarked',
                                  'who'])])

In [9]:
preprocessor = ColumnTransformer([

                                  ('num_pipeline',num_preprocessor,num_feature),
                                  ('cat_pipeline',cat_preprocessor,cat_feature)

                                 ],remainder='passthrough'
                                )
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('num_pipeline',
                                 ColumnTransformer(transformers=[('num_transformer',
                                                                  Pipeline(steps=[('imputer',
                                                                                   SimpleImputer(strategy='median')),
                                                                                  ('scaler',
                                                                                   RobustScaler())]),
                                                                  ['Age',
                                                                   'Fare'])]),
                                 ['Age', 'Fare']),
                                ('cat_pipeline',
                                 ColumnTransformer(remainder='passthrough',
                                                   transformers=[('cat_to_ohe',
                                                                  Pipeline(steps=[('imputer',
                                                                                   SimpleImputer(strategy='most_frequent')),
                                                                                  ('OHE',
                                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                                 sparse=False))]),
                                                                  ['Pclass',
                                                                   'SibSp',
                                                                   'Parch',
                                                                   'Embarked',
                                                                   'who'])]),
                                 ['Pclass', 'SibSp', 'Parch', 'Embarked',
                                  'who'])])

In [10]:
pd.DataFrame(preprocessor.fit_transform(X))

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,-0.461538,-0.311813,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,0.769231,2.459687,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,-0.153846,-0.282598,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.538462,1.672674,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.538462,-0.277188,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884,-0.076923,-0.062941,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
885,-0.692308,0.672856,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
886,0.000000,0.389358,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
887,-0.153846,0.672856,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
